# Westeros Tutorial Part II - Introducing emissions

In the first part, the model chose not base the power system on wind power as electricity from  wind turbines was more expensive than electricity produced from coal. However, now we introduce emissions to investigate the impact of climate policy. Let's see what happens then.

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform(dbtype='HSQLDB')

In [ ]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'emission_bound','introducing an upper bound on emissions',
                  keep_solution=False)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Introducing Emissions

In [ ]:
# first we introduce the emission specis CO2 and the emission category GHG
scen.add_set('emission', 'CO2')
scen.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'USD/GWa',
}

emission_factor = make_df(base_emission_factor, technology= 'coal_ppl', emission= 'CO2', value = 100.)
scen.add_par('emission_factor', emission_factor)

## Define a Bound on Emissions

The `type_year: cumulative` assigns an upper bound on the *average emissions per year* over the entire time horizon.

In [ ]:
scen.add_par('bound_emission', key=[country, 'GHG', 'all', 'cumulative'],
             val=4000., unit='tCO2')

## Time to Solve the Model

In [ ]:
scen.commit(comment='introducing emissions and setting an upper bound')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Capacity

Given how many new plants are built, how many are actually used?

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Electricity Price

And how much does the electricity cost? These prices are taken from the **dual variables** of the solution and are given the name **shadow prices**. They reflect the marginal price of electricity, taken from the most expensive producer.  

Note that the price drop when the most expensive technology is no longer in the system.

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

## Close the connection to the database

In [ ]:
mp.close_db()